# DT Assignment1

# Data Loading

In [9]:
import pandas as pd 
import numpy as np

In [10]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
def get_gini(df, label):
    gini = pd.DataFrame(df[label].value_counts()).apply(lambda x: 1 - sum((x / sum(x)) ** 2)).values[0] #label 클래스에 대해서 클래스 각각 개수에 대한 gini계수 공식을 활용하여  
    return gini #gini를 return하도록 만들었습니다. 

In [12]:
print(get_gini(pd_data,'class_buys_computer')) #기존의 값과 동일하게 나타나는 것을 알 수 있습니다. 
print(get_gini(pd_data,'age')) #추가적으로  age에 대해서도 확인해 보았습니다. 

0.4591836734693877
0.6632653061224489


# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [13]:
def get_binary_split(df, attribute):
    from itertools import combinations
    
    class_num = len(set(df[attribute].values)) #각 attribue의 values를 집합으로 지정하고 그 갯수를 저장하였습니다.  
    result = [] #빈 리스트를 만들어 주었습니다. 
    
    for i in range(1, class_num): #원소 하나부터 전체 라벨 - 1개로 묶인 리스트들을 만들어서 result에 추가시켜주었습니다.
        result.extend(list(combinations(set(df[attribute].values), r=i))) #combinations 결과 전체가 리스트로 묶여 나오기 때문에 풀어주기 위하여 extend를 사용하였습니다.
    result = list(map(list, result)) #리스트내의 튜플들을 리스트내의 리스트로 만들어 주었습니다. 
    
    return result

In [14]:
get_binary_split(pd_data,"age") #동일하게 출력되는 것을 확인할 수 있습니다. 

[['middle_aged'],
 ['senior'],
 ['youth'],
 ['middle_aged', 'senior'],
 ['middle_aged', 'youth'],
 ['senior', 'youth']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [15]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    cases = get_binary_split(df, attribute) #위에서 만든 calss에 대한 이진분류기를 활용하여 저장하였습니다.
    class_names = set(df[attribute])
    
    for case in cases:
        a = df.loc[df[attribute].isin(case), :]
        b = df.loc[df[attribute].isin(class_names - set(case)), :]
        
        gini = (get_gini(a, label) * len(a.index) +  get_gini(b, label) * len(b.index)) / (len(df.index)) #지니계수 공식 사용 
        result.update({"_".join(case): gini})
    
    return result

In [16]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth': 0.39365079365079364,
 'middle_aged_senior': 0.39365079365079364,
 'middle_aged_youth': 0.4571428571428572,
 'senior_youth': 0.35714285714285715}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [17]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items()) #동일하게 출력되었습니다. 

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [18]:
##문제1 답안

get_binary_split(pd_data, "income") #변수 income에 대한 이진분류가 출력되는 것을 확인할 수 있습니다. 

[['high'],
 ['low'],
 ['medium'],
 ['high', 'low'],
 ['high', 'medium'],
 ['low', 'medium']]

In [20]:
##문제2 답안

def min_gini_searcher(df, label):
    min_gini_index_dict = {}
    min_gini_index = None

    for index, col_name in enumerate(df.columns):
        if col_name != label:   
            gini_indices_dict = get_attribute_gini_index(df, col_name, label)
            if len(gini_indices_dict) >= 1:
                if min_gini_index is None:
                    min_gini_index = min(gini_indices_dict.items())[1]
                    min_gini_index_dict = {col_name: min(gini_indices_dict.items())}
                elif index >= 1 and min(gini_indices_dict.items())[1] < min_gini_index:
                    min_gini_index = min(gini_indices_dict.items())[1]
                    min_gini_index_dict = {col_name: min(gini_indices_dict.items())}
    return min_gini_index_dict

min_gini_searcher(pd_data, "class_buys_computer")


{'age': ('middle_aged', 0.35714285714285715)}

출력되는 결과를 통하여 변수 age의 ("middle_aged"), ("youth", "senior") 조합으로 이진 분류하는 것이 가장 낮은 gini index 값을 가지므로 변수 age가 분류에 있어 가장 중요한 변수라는 것을 알 수 있습니다. 또한 해당 변수의 최소 지니 계수 값은 0.35714285714285715로 출력되는 것을 확인할 수 있습니다. 

In [23]:
##문제3 답안

splited_df_1 = pd_data.loc[pd_data["age"].isin(["youth", "senior"]), :]
splited_df_2 = pd_data.loc[pd_data["age"] == "middle_aged", :]

min_gini_searcher(splited_df_1, "class_buys_computer")

{'student': ('no', 0.31999999999999984)}

In [15]:
min_gini_searcher(splited_df_2, "class_buys_computer")

{'income': ('high', 0.0)}

위의 결과를 통하여 스플릿된 첫 번째 데이터프레임에서는 변수 student의 ("no"), ("yes") 조합으로 이진 분류하는 것이 0.31999999999999984로 가장 낮은 gini index 값을 가지며 스플릿된 두 번째 데이터프레임에서는 변수 income의 ("high"), ("medium", "low") 조합으로 이진 분류하는 것이 0.0으로 가장 낮은 gini index 값을 가지는 것을 확인할 수 있었습니다. 그러므로 전자의 경우는 변수 age가, 후자의 경우는 변수 student가 가장 중요한 변수라고 판단하였습니다. 단 후자의 경우에는 이미 종속 변수의 값이 전부 같으므로 어떻게 분류하더라도 gini index가 0이 나올 것이라고 추측할 수 있습니다.